# Signal Muting

This notebook demonstrates the signal muting functionality of SHF+ devices.
Signal muting can be used to effectively reduce the noise on idle signal lines.

### 0. Python Imports

Import the necessary DSL classes from LabOne Q

In [8]:
from laboneq.simple import *
from laboneq.contrib.example_helpers.generate_example_datastore import (
    generate_example_datastore,
    get_first_named_entry,
)


### 1. Device Setup

Create the device setup instance for the SHFQC instrument.


In [9]:
# Build an in-memory data store with device setup and qubit parameters for the
# example notebooks
dummy_db = generate_example_datastore(in_memory=True)

use_emulation = True


In [10]:
# Choose your setup - local feedback on a standalone SHFQC or ...
# feedback_type="local"
# my_setup = get_first_named_entry(db=dummy_db, name="6_qubit_setup_shfqc_calibrated")
# ... global feedback on a combination of SHFSG and SHFQA, connected through a PQSC
feedback_type = "global"
my_setup = get_first_named_entry(
    db=dummy_db, name="6_fixed_qubit_setup_shfqc_calibrated"
)


In [11]:
session = Session(my_setup)
session.connect(do_emulation=True, ignore_version_mismatch=True)

[2024.05.13 15:34:53.457] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is c:\Users\thomass\Programming\laboneq_develop\qccs\examples\00_reference\laboneq_output\log
[2024.05.13 15:34:53.464] INFO    VERSION: laboneq 2.30.0b20240508120547+8ccbb053
[2024.05.13 15:34:53.468] INFO    Connecting to data server at localhost:8004
[2024.05.13 15:34:53.480] INFO    Connected to Zurich Instruments LabOne Data Server version 24.04 at localhost:8004
[2024.05.13 15:34:53.486] INFO    Configuring the device setup
[2024.05.13 15:34:53.488] INFO    The device setup is configured


### 2. `automute` Calibration Option

Define the calibration function for the `acquire` and `measure` line of the SHFQA.

* The muting functionality is enabled on the `measure` line with the option `automute`.

In [13]:
q0 = my_setup.logical_signal_groups["q0"]

q0.logical_signals["acquire_line"].calibration = SignalCalibration(
    oscillator=Oscillator(modulation_type=ModulationType.SOFTWARE, frequency=-2.5e8),
    local_oscillator=Oscillator(frequency=2e9),
    range=5,
)

q0.logical_signals["measure_line"].calibration = SignalCalibration(
    oscillator=Oscillator(modulation_type=ModulationType.SOFTWARE, frequency=-2.5e8),
    local_oscillator=Oscillator(frequency=2e9),
    automute=True,  # <-- Mute this output line when not playing pulses
    range=10,
)

### 3. Experiment

Define an experiment that makes use of the muting functionality.

* The small amplitude of the play pulse operation allows us see the effect of the muting function better.
* While muting was already enabled by the calibration option `automute`, this functionality needs a sufficiently long delay in between pulses to become active.
* More specifically, a minimum delay of 280 ns is needed to mute a given signal line after a pulse has been played on it.

In [14]:
exp = Experiment(
    signals=[
        ExperimentSignal("acquire", map_to=q0.logical_signals["acquire_line"]),
        ExperimentSignal("measure", map_to=q0.logical_signals["measure_line"]),
    ]
)
pulse = pulse_library.const(length=5e-7, amplitude=1)

with exp.acquire_loop_rt(count=2**5):
    with exp.section(uid="measure"):
        exp.play("measure", pulse=pulse, amplitude=0.01)
        exp.acquire("acquire", kernel=pulse, handle="h")
        exp.delay("acquire", 280e-9)
        exp.delay("measure", 280e-9)  # <-- muting active for delays >= 280 ns

compiled_exp = session.compile(exp)
print("\nSeqC:\n")
print(compiled_exp.scheduled_experiment.artifacts.src[0]["text"])

[2024.05.13 15:35:15.222] INFO    Starting LabOne Q Compiler run...
[2024.05.13 15:35:15.232] INFO    Schedule completed. [0.001 s]
[2024.05.13 15:35:15.254] INFO    Code generation completed for all AWGs. [0.019 s]
[2024.05.13 15:35:15.258] INFO    Completed compilation step 1 of 1. [0.026 s]
[2024.05.13 15:35:15.285] INFO     ───────────────────────────────────────────────────────────── 
[2024.05.13 15:35:15.288] INFO      Device    AWG   SeqC LOC   CT entries   Waveforms   Samples  
[2024.05.13 15:35:15.292] INFO     ───────────────────────────────────────────────────────────── 
[2024.05.13 15:35:15.295] INFO      shfqc_0     0         12            0           1      2016  
[2024.05.13 15:35:15.297] INFO     ───────────────────────────────────────────────────────────── 
[2024.05.13 15:35:15.299] INFO      TOTAL                 12            0                  2016  
[2024.05.13 15:35:15.302] INFO     ───────────────────────────────────────────────────────────── 
[2024.05.13 15:35:1

### 4. Result

We are now ready to run the experiment and observe the muting of the signal on an oscilloscope.

The following wiring to the oscilloscope is used
* QA output signal
* QA marker signal


In [15]:
_ = session.run(compiled_exp)

[2024.05.13 15:35:18.260] INFO    Configuring the device setup
[2024.05.13 15:35:18.263] INFO    The device setup is configured
[2024.05.13 15:35:18.278] WARNING SHFQC/QA:dev12003: Device output muting is enabled, but the device is not SHF+ and therefore no muting will happen. It is suggested to disable it.
[2024.05.13 15:35:18.284] INFO    Starting near-time execution...
[2024.05.13 15:35:18.345] INFO    Finished near-time execution.


We see the behavior of the QA output and marker signal before and during the sequence of pulses played by
```
    exp.play("measure", pulse=pulse, amplitude=0.01)
```

and observe the following

* Without muting, we see the same noise level before and during the sequence of pulses 

* We see the muting function being active from the marker signal.

* When muting is active the noise level between individual pulses is reduced compared to before the pulse sequence.
